In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../'))

In [4]:
import pandas as pd
import warnings

import json
from numpy import random
from dataclasses import dataclass

from model.utils import report_results
from model.gradient_boosting import train_gradient_boosting

from sklearn.ensemble import HistGradientBoostingClassifier

DEFAULT_RANDOM_SEED = 774
random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)
seed_list = random.random_integers(low=0, high=2**32 - 1, size=100)
warnings.filterwarnings("ignore")

search_params = {"learning_rate": (0.05, 0.1, 0.5, 1), "max_features": (0.05, 0.1, 0.15, 0.2, 0.5, "sqrt"), "l2_regularization": (0, 0.5, 1), "max_depth": (16, 32, 64, None), "class_weight": ("balanced",)}

/var/folders/vp/tqkr07mj3cn274npl2jmvy5w0000gq/T/ipykernel_2228/114175797.py:15: DeprecationWarning: This function is deprecated. Please call randint(0, 4294967295 + 1) instead
  seed_list = random.random_integers(low=0, high=2**32 - 1, size=100)


In [5]:
@dataclass
class RunConfiguration:
  run_grid_search: bool
  default_parameters: dict

In [6]:
def get_parameters(df: pd.DataFrame, run_config: RunConfiguration):
  if not run_config.run_grid_search:
    return run_config.default_parameters
  
  grid_search_response = train_gradient_boosting(target="subtype", data=df, grid_search_params=search_params)
  parameters = {k: grid_search_response.model.get_params()[k] for k in grid_search_response.model.get_params().keys() & search_params.keys() }
  print(parameters)
  return parameters

def run_tests(category: str, pvalues: str, select_genes, not_biased_config: RunConfiguration, biased_config: RunConfiguration = None):
  data = pd.read_csv(f"../../preprocessed/{category}/genes.csv").drop(columns=["sample_id"])
  pvalues = json.loads(open(f"../../preprocessed/{category}/important_genes_{pvalues}_pvalue.json").readline())

  chosen_genes_all = select_genes(pvalues)
  print(f"Total chosen genes: {len(chosen_genes_all)}")

  print("Reporting not biased results")
  df = data[["subtype", "sex", *chosen_genes_all]]
  print(report_results(df, HistGradientBoostingClassifier, get_parameters(df, not_biased_config), seed_list).report)

  if biased_config:
    print("Reporting biased results")
    chosen_genes_male = list(set([y["gene"] for x in [subtype_items["Male"][:50] for subtype_items in pvalues.values()] for y in x]))
    df = data[["subtype", "sex", *chosen_genes_male]]
    print(report_results(df, HistGradientBoostingClassifier, get_parameters(df, biased_config), seed_list).report)

    chosen_genes_female = list(set([y["gene"] for x in [subtype_items["Female"][:50] for subtype_items in pvalues.values()] for y in x]))
    df = data[["subtype", "sex", *chosen_genes_female]]
    print(report_results(df, HistGradientBoostingClassifier, get_parameters(df, biased_config), seed_list).report)

# TTest 25 genes per sex-subtype

In [7]:
run_tests(
  category="min_tpm_5",
  pvalues="ttest",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': 64, 'l2_regularization': 0}
  ),
  biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': None, 'l2_regularization': 1}
  )
)

Total chosen genes: 375
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 32, 'learning_rate': 0.1, 'max_features': 0.05, 'class_weight': 'balanced'}


100%|██████████| 100/100 [16:43<00:00, 10.03s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8814 ± 0.0240  0.8604 ± 0.0349  0.9033 ± 0.0329
1      F1 (Macro)  0.8505 ± 0.0342  0.8141 ± 0.0528  0.8749 ± 0.0467
2  Recall (Macro)  0.8374 ± 0.0342  0.8053 ± 0.0492  0.8667 ± 0.0461
3         ROC AUC  0.9891 ± 0.0058  0.9866 ± 0.0067  0.9923 ± 0.0089
4        Accuracy  0.8854 ± 0.0227  0.8701 ± 0.0329  0.9104 ± 0.0293
5        Duration  9.2884 ± 0.2871                0                0
Reporting biased results


{'l2_regularization': 0.5, 'max_depth': 16, 'learning_rate': 0.1, 'max_features': 0.05, 'class_weight': 'balanced'}


100%|██████████| 100/100 [11:19<00:00,  6.79s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8737 ± 0.0258  0.8634 ± 0.0352  0.8863 ± 0.0325
1      F1 (Macro)  0.8388 ± 0.0384  0.8230 ± 0.0524  0.8525 ± 0.0464
2  Recall (Macro)  0.8315 ± 0.0380  0.8195 ± 0.0488  0.8420 ± 0.0447
3         ROC AUC  0.9877 ± 0.0067  0.9881 ± 0.0066  0.9888 ± 0.0109
4        Accuracy  0.8819 ± 0.0242  0.8701 ± 0.0320  0.8955 ± 0.0296
5        Duration  6.1552 ± 0.1253                0                0


{'l2_regularization': 0.5, 'max_depth': None, 'learning_rate': 0.05, 'max_features': 0.05, 'class_weight': 'balanced'}


100%|██████████| 100/100 [11:31<00:00,  6.92s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8655 ± 0.0262  0.8385 ± 0.0398  0.8898 ± 0.0337
1      F1 (Macro)  0.8247 ± 0.0360  0.7742 ± 0.0528  0.8569 ± 0.0481
2  Recall (Macro)  0.8119 ± 0.0348  0.7739 ± 0.0500  0.8472 ± 0.0456
3         ROC AUC  0.9850 ± 0.0059  0.9817 ± 0.0080  0.9894 ± 0.0077
4        Accuracy  0.8681 ± 0.0252  0.8442 ± 0.0380  0.8955 ± 0.0301
5        Duration  6.2660 ± 0.1564                0                0


## TTest overall (without grouping by sex), 45 genes per subtype

In [8]:
run_tests(
  category="min_tpm_5",
  pvalues="ttest_overall",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [subtype_items[:45] for subtype_items in pvalues.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': 64, 'l2_regularization': 0.5}
  ),
  biased_config=None
)

Total chosen genes: 384
Reporting not biased results
{'l2_regularization': 0, 'max_depth': None, 'learning_rate': 0.05, 'max_features': 0.15, 'class_weight': 'balanced'}


100%|██████████| 100/100 [22:55<00:00, 13.76s/it]

           Metric           Overall             Male           Female
0   F1 (Weighted)   0.8651 ± 0.0260  0.8501 ± 0.0381  0.8892 ± 0.0335
1      F1 (Macro)   0.8276 ± 0.0365  0.7906 ± 0.0547  0.8545 ± 0.0480
2  Recall (Macro)   0.8143 ± 0.0353  0.7920 ± 0.0511  0.8472 ± 0.0454
3         ROC AUC   0.9859 ± 0.0055  0.9853 ± 0.0077  0.9876 ± 0.0075
4        Accuracy   0.8750 ± 0.0245  0.8571 ± 0.0351  0.8955 ± 0.0301
5        Duration  13.0852 ± 0.3599                0                0


## TTest first 400 genes by pvalue

In [9]:
run_tests(
  category="min_tpm_5",
  pvalues="ttest",
  select_genes=lambda pvalues: list(set([x["gene"] for x in sorted([z for x in pvalues.values() for y in x.values() for z in y], key=lambda x: x["pvalue"])]))[:400],
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.1, 'learning_rate': 0.1, 'max_depth': 32, 'l2_regularization': 0}
  ),
  biased_config=None
)

Total chosen genes: 400
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 32, 'learning_rate': 0.1, 'max_features': 0.1, 'class_weight': 'balanced'}


100%|██████████| 100/100 [18:29<00:00, 11.10s/it]

           Metric           Overall             Male           Female
0   F1 (Weighted)   0.8385 ± 0.0266  0.8140 ± 0.0368  0.8643 ± 0.0357
1      F1 (Macro)   0.8003 ± 0.0362  0.7604 ± 0.0498  0.8245 ± 0.0502
2  Recall (Macro)   0.7891 ± 0.0342  0.7598 ± 0.0462  0.8153 ± 0.0462
3         ROC AUC   0.9767 ± 0.0074  0.9712 ± 0.0105  0.9847 ± 0.0100
4        Accuracy   0.8472 ± 0.0259  0.8182 ± 0.0359  0.8657 ± 0.0331
5        Duration  10.5525 ± 0.4567                0                0


## TTest overall first 400 genes by pvalue

In [10]:
run_tests(
  category="min_tpm_5",
  pvalues="ttest_overall",
  select_genes=lambda pvalues: list(set([x["gene"] for x in sorted([x for subtype_items in pvalues.values() for x in subtype_items], key=lambda x: x["pvalue"])]))[:400],
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.15, 'learning_rate': 0.05, 'max_depth': 64, 'l2_regularization': 0.5}
  ),
  biased_config=None
)

Total chosen genes: 400
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 16, 'learning_rate': 0.1, 'max_features': 0.1, 'class_weight': 'balanced'}


100%|██████████| 100/100 [17:41<00:00, 10.61s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8410 ± 0.0256  0.8143 ± 0.0368  0.8621 ± 0.0349
1      F1 (Macro)  0.7978 ± 0.0357  0.7646 ± 0.0495  0.8199 ± 0.0494
2  Recall (Macro)  0.7901 ± 0.0340  0.7640 ± 0.0456  0.8139 ± 0.0472
3         ROC AUC  0.9774 ± 0.0081  0.9709 ± 0.0107  0.9856 ± 0.0102
4        Accuracy  0.8472 ± 0.0247  0.8182 ± 0.0352  0.8657 ± 0.0321
5        Duration  9.9234 ± 0.4147                0                0


In [11]:
run_tests(
  category="min_tpm_5",
  pvalues="wilcoxon",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.2, 'learning_rate': 0.1, 'max_depth': 16, 'l2_regularization': 1}
  )
)

Total chosen genes: 411
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 32, 'learning_rate': 0.05, 'max_features': 0.1, 'class_weight': 'balanced'}


100%|██████████| 100/100 [23:13<00:00, 13.93s/it]

           Metric           Overall             Male           Female
0   F1 (Weighted)   0.8860 ± 0.0220  0.8769 ± 0.0298  0.8926 ± 0.0294
1      F1 (Macro)   0.8597 ± 0.0290  0.8483 ± 0.0413  0.8618 ± 0.0396
2  Recall (Macro)   0.8468 ± 0.0292  0.8416 ± 0.0393  0.8562 ± 0.0385
3         ROC AUC   0.9894 ± 0.0048  0.9897 ± 0.0051  0.9912 ± 0.0072
4        Accuracy   0.8889 ± 0.0210  0.8831 ± 0.0273  0.8955 ± 0.0281
5        Duration  13.2317 ± 0.3895                0                0


In [12]:
run_tests(
  category="min_tpm_5",
  pvalues="cramervonmises",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': None, 'l2_regularization': 0}
  )
)

Total chosen genes: 358
Reporting not biased results
{'l2_regularization': 0.5, 'max_depth': 16, 'learning_rate': 0.1, 'max_features': 0.1, 'class_weight': 'balanced'}


100%|██████████| 100/100 [05:56<00:00,  3.56s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8781 ± 0.0219  0.8693 ± 0.0301  0.8838 ± 0.0303
1      F1 (Macro)  0.8403 ± 0.0322  0.8251 ± 0.0457  0.8461 ± 0.0432
2  Recall (Macro)  0.8280 ± 0.0308  0.8185 ± 0.0410  0.8434 ± 0.0403
3         ROC AUC  0.9871 ± 0.0058  0.9870 ± 0.0065  0.9881 ± 0.0083
4        Accuracy  0.8819 ± 0.0209  0.8831 ± 0.0279  0.8955 ± 0.0278
5        Duration  2.3525 ± 1.6347                0                0


In [ ]:
run_tests(
  category="min_tpm_5",
  pvalues="brunnermunzel",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=False,
    default_parameters={'learning_rate': 0.05, 'l2_regularization': 0.5, 'max_depth': 64, 'max_features': 0.2, 'class_weight': 'balanced'}
  )
)

Total chosen genes: 379
Reporting not biased results
{'learning_rate': 0.05, 'l2_regularization': 0.5, 'max_depth': 64, 'max_features': 0.2, 'class_weight': 'balanced'}


100%|██████████| 100/100 [02:47<00:00,  1.67s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8947 ± 0.0220  0.8935 ± 0.0289  0.8934 ± 0.0319
1      F1 (Macro)  0.8708 ± 0.0301  0.8665 ± 0.0437  0.8673 ± 0.0414
2  Recall (Macro)  0.8654 ± 0.0315  0.8582 ± 0.0451  0.8656 ± 0.0416
3         ROC AUC  0.9911 ± 0.0044  0.9919 ± 0.0055  0.9910 ± 0.0062
4        Accuracy  0.8958 ± 0.0215  0.8961 ± 0.0273  0.8955 ± 0.0312
5        Duration  1.5309 ± 0.0832                0                0


In [13]:
run_tests(
  category="min_tpm_5",
  pvalues="epps_singleton",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': 32, 'l2_regularization': 0}
  )
)

Total chosen genes: 399
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 64, 'learning_rate': 0.05, 'max_features': 0.1, 'class_weight': 'balanced'}


100%|██████████| 100/100 [09:08<00:00,  5.49s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8941 ± 0.0221  0.8770 ± 0.0331  0.9068 ± 0.0291
1      F1 (Macro)  0.8631 ± 0.0298  0.8410 ± 0.0458  0.8788 ± 0.0389
2  Recall (Macro)  0.8569 ± 0.0306  0.8420 ± 0.0458  0.8694 ± 0.0397
3         ROC AUC  0.9908 ± 0.0041  0.9909 ± 0.0053  0.9924 ± 0.0064
4        Accuracy  0.8958 ± 0.0212  0.8831 ± 0.0302  0.9104 ± 0.0279
5        Duration  5.2926 ± 0.2129                0                0


In [14]:
run_tests(
  category="min_tpm_5",
  pvalues="anderson_ksamp",
  select_genes=lambda pvalues: list(set([y["gene"] for x in [sex_values[:25] for subtype_items in pvalues.values() for sex_values in subtype_items.values()] for y in x])),
  not_biased_config=RunConfiguration(
    run_grid_search=True,
    default_parameters={'max_features': 0.05, 'learning_rate': 0.1, 'max_depth': 64, 'l2_regularization': 0}
  )
)

Total chosen genes: 242
Reporting not biased results
{'l2_regularization': 0, 'max_depth': 64, 'learning_rate': 0.05, 'max_features': 0.05, 'class_weight': 'balanced'}


100%|██████████| 100/100 [05:49<00:00,  3.50s/it]

           Metric          Overall             Male           Female
0   F1 (Weighted)  0.8576 ± 0.0266  0.8474 ± 0.0397  0.8722 ± 0.0347
1      F1 (Macro)  0.8265 ± 0.0366  0.8171 ± 0.0560  0.8329 ± 0.0482
2  Recall (Macro)  0.8208 ± 0.0353  0.8137 ± 0.0539  0.8260 ± 0.0455
3         ROC AUC  0.9840 ± 0.0073  0.9817 ± 0.0105  0.9879 ± 0.0102
4        Accuracy  0.8611 ± 0.0257  0.8571 ± 0.0370  0.8806 ± 0.0325
5        Duration  3.3159 ± 0.1529                0                0
